In [20]:
# load packages
import csv
from pathlib import Path
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [21]:
# read data
base_path = Path.cwd()
edited_path = base_path / "Olympic" / "edited"
dim_path = edited_path /'dim.csv'
fact_path = edited_path /'fact.csv'
dim_data = pd.read_csv(dim_path)
fact_data = pd.read_csv(fact_path)

In [28]:
# transform data into what we need
fact_data_cleaned = fact_data.dropna(subset=['gdpkey'])[['medaltype', 'gdpkey']]
fact_data_cleaned['gdpkey'] = fact_data_cleaned['gdpkey'].astype(int)
merged_data_cleaned = pd.merge(fact_data_cleaned, dim_data, on='gdpkey')
final_data = merged_data_cleaned.drop('gdpkey', axis=1)
final_data = final_data[final_data['medaltype'] == 'GOLD']
final_data = final_data.groupby('gdppercent').size().reset_index(name='gold_count')
final_data

,gdppercent,gold_count
0,10,1
1,20,4
2,30,13
3,40,22
4,50,42
5,60,98
6,70,205
7,80,322
8,90,523
9,100,3225


In [33]:
# get the poisson regression by modeling
from statsmodels.formula.api import glm
import statsmodels.api as sm

poisson_model = glm('gold_count ~ gdppercent', data=final_data, family=sm.families.Poisson()).fit()
poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             gold_count   No. Observations:                   10
Model:                            GLM   Df Residuals:                        8
Model Family:                 Poisson   Df Model:                            1
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -281.92
Date:                Sun, 31 Mar 2024   Deviance:                       504.42
Time:                        10:38:30   Pearson chi2:                     495.
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.9266      0.146    -13.211      0.000      -2.212      -1.641
gdppercent     0.0986      0.002     63.940      0.000       0.096       0.102
==============================================================================
"""